# **PersonalRank Recommender**

## **Import Data**

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My\ Drive/Web mining project

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1o8pv34povS0R5_x0ABuISxxGG3LxQ3ET/Web mining project


In [4]:
import pandas as pd
import operator
import pickle
from scipy.sparse.linalg import gmres
from scipy.sparse import csr_matrix
import numpy as np
import math

In [17]:
user_friends_df = pd.read_table('data/dataset/user_friends.dat')
df_train = pd.read_csv('data/split/train_user_artists.csv')
df_train_tune = pd.read_csv('data/split/train_tune_user_artists.csv')
df_train_tune = df_train_tune[['artistID', 'userID', 'weight']]
df_val = pd.read_csv('data/split/val_user_artists.csv')
df_test = pd.read_csv('data/split/test_user_artists.csv')
tag_train = pd.read_csv('data/tags/correlation_tags_train_0.4_average.csv')

## **User Similarity Computation**

In [12]:
#Function to calculate the similarity between users

#Represent each user with tfidf value
def user_tfidf(user_friends_df,  df_train):
  tf = dict()
  idf = dict()
  tfidf = dict()
  count = dict()
  artist_con = dict()
  user_list = []
  artist_list = []
  for i in range(len(df_train)):
    user,artist,weight=str(df_train['userID'][i]),"artist_"+str(df_train['artistID'][i]),df_train['weight'][i]
    if user not in user_list:
      user_list.append(user)
    if artist not in artist_list:
      artist_list.append(artist)
      artist_con[artist] = 0
    if user not in count:
      count[user] = dict()
    count[user][artist] = weight
    artist_con[artist] += 1
  for user in user_list:
    tf[user] = dict()
    idf[user] = dict()
    tfidf[user] = dict()
    for artist in count[user]:
      tf[user][artist] = count[user][artist]  / sum(count[user].values())
      idf[user][artist] = math.log(len(artist_list) / artist_con[artist])
      tfidf[user][artist] = tf[user][artist] * idf[user][artist]
  return tfidf

#Calculate cosine similarity based on tfidf values
def user_cosine_similarity(user1, user2, tfidf):
  nume = 0
  sum_user1 = 0
  sum_user2 = 0
  for artist in tfidf[user1].keys() & tfidf[user2].keys():
    nume += tfidf[user1][artist] * tfidf[user2][artist]
  for value in tfidf[user1].values():
    sum_user1 += pow(value,2)
  for value in tfidf[user2].values():
    sum_user2 += pow(value,2)
  similarity = nume / (pow(sum_user1, 0.5) * pow(sum_user2, 0.5))
  return similarity

In [13]:
#calculate similarity and save it as similarity.pickle
tfidf = user_tfidf(user_friends_df,  df_train)
user_list = []
similarity = {}
for i in range(len(df_train)):
  user,artist,weight=str(df_train['userID'][i]),"artist_"+str(df_train['artistID'][i]),df_train['weight'][i]
  if user not in user_list:
    user_list.append(user)
for user in user_list:
  similarity[user] = dict()
  for user1 in user_list:
    similarity[user][user1] = user_cosine_similarity(user, user1, tfidf)

#the similarity matrix have been saved in data/interim/similarity.pickle
with open('data/interim/similarity.pickle', 'wb') as f:
  f.write(pickle.dumps(similarity))

## **Graph Construction**

### Without tag

In [33]:
similarity = pickle.loads(open('data/interim/similarity.pickle', "rb").read())
k_friend = 0.3
k_artist = 1

#build random walk graph
def buildGrapha_without_tag(df_train, user_friends_df, k_friend, k_artist, similarity):
  graph = dict()
  weights = dict()
  friends = dict()
  user_list = []
  tags = dict()
  tag_list = []
  
  #user_artist data
  for i in range(len(df_train)):
    user,artist,weight=str(df_train['userID'][i]),"artist_"+str(df_train['artistID'][i]),df_train['weight'][i]
    if user not in user_list:
      user_list.append(user)
    if user not in graph:
      graph[user] = dict()
      weights[user] = weight
    else:
      weights[user] += weight
    graph[user][artist] = weight
    if artist not in graph:
      graph[artist]=dict()
      weights[artist] = weight
    else:
      weights[artist] += weight
    graph[artist][user] = weight
  
  #transform the weight of edges between user and artist
  for users in user_list:
    for artists in graph[users]:
      # print(users + artist)
      tmp = graph[users][artists]
      graph[users][artists] = k_artist * tmp / (weights[users] * 1.0)
      graph[artists][users] = k_artist * tmp / (weights[artists] * 1.0)

  #user_friend data
  for i in range(len(user_friends_df)):
    user, friend = str(user_friends_df['userID'][i]), str(user_friends_df['friendID'][i])
    if user not in graph:
      graph[user] = dict()
    if user not in friends:
      friends[user] = []
    graph[user][friend] = 0.5 + 0.5 * similarity[user][friend]
    friends[user].append(friend)

  #transform the weight of edges between users and friends
  for user,friend_list in friends.items():
    for friend in friend_list:
      graph[user][friend] = k_friend * graph[user][friend] / (1.0 * len(friend_list))


  for point, point_dic in graph.items():
    weights_sum = sum(graph[point].values())   
    for next_point in graph[point]:
      if weights_sum != 0:
        graph[point][next_point] = graph[point][next_point] / weights_sum
  
  return graph

### With Tag

In [14]:
similarity = pickle.loads(open('data/interim/similarity.pickle', "rb").read())
k_friend = 0.3
k_tag = 0.1
k_artist = 1

#build random walk graph
def buildGrapha_with_tag(df_train, tag_train, user_friends_df, k_friend, k_tag, k_artist, similarity):
  graph = dict()
  weights = dict()
  friends = dict()
  user_list = []
  tags = dict()
  tag_list = []
  
  #user_artist data
  for i in range(len(df_train)):
    user,artist,weight=str(df_train['userID'][i]),"artist_"+str(df_train['artistID'][i]),df_train['weight'][i]
    if user not in user_list:
      user_list.append(user)
    if user not in graph:
      graph[user] = dict()
      weights[user] = weight
    else:
      weights[user] += weight
    graph[user][artist] = weight
    if artist not in graph:
      graph[artist]=dict()
      weights[artist] = weight
    else:
      weights[artist] += weight
    graph[artist][user] = weight
  
  #transform the weight of edges between user and artist
  for users in user_list:
    for artists in graph[users]:
      # print(users + artist)
      tmp = graph[users][artists]
      graph[users][artists] = k_artist * tmp / (weights[users] * 1.0)
      graph[artists][users] = k_artist * tmp / (weights[artists] * 1.0)

  #user_friend data
  for i in range(len(user_friends_df)):
    user, friend = str(user_friends_df['userID'][i]), str(user_friends_df['friendID'][i])
    if user not in graph:
      graph[user] = dict()
    if user not in friends:
      friends[user] = []
    graph[user][friend] = 0.5 + 0.5 * similarity[user][friend]
    friends[user].append(friend)

  #transform the weight of edges between users and friends
  for user,friend_list in friends.items():
    for friend in friend_list:
      graph[user][friend] = k_friend * graph[user][friend] / (1.0 * len(friend_list))


  #user_tag_artist data

  for i in range(len(tag_train)):
    user, artist,	tag = str(tag_train['userID'][i]),"artist_"+str(tag_train['artistID'][i]),"tag_" + str(tag_train['cluster'][i])
    if tag not in tag_list:
      tag_list.append(tag)
    if user not in graph:
      graph[user] = dict()
    if user not in tags:
      tags[user] = []
    if tag not in graph:
      graph[tag] = dict()
    if tag not in tags:
      tags[tag] = []
    if artist not in graph:
      graph[artist] = dict()
    if artist not in tags:
      tags[artist] = []
    if tag not in graph[user]:
      graph[user][tag] = 0
      graph[tag][user] = 0
    if artist not in graph[tag]:
      graph[tag][artist] = 0
      graph[artist][tag] = 0
    tags[user].append(tag)
    tags[tag].append(user)
    tags[tag].append(artist)
    tags[artist].append(tag)
    graph[user][tag] += 1
    graph[tag][user] += 1
    graph[tag][artist] += 1
    graph[artist][tag] += 1
  
  #transform the weight of edges between users, tags and artists
  for tag in tag_list:
    for links in graph[tag]:
      graph[links][tag] = k_tag * graph[links][tag] / (1.0 * len(tags[links]))
      graph[tag][links] = k_tag * graph[tag][links] / (1.0 * len(tags[tag]))

  for point, point_dic in graph.items():
    weights_sum = sum(graph[point].values())   
    for next_point in graph[point]:
      if weights_sum != 0:
        graph[point][next_point] = graph[point][next_point] / weights_sum
  
  return graph

## **Implementation**

### Based on Iteration

In [15]:
#Personal_rank algorithm based on iteration 
def personal_rank(graph,root,alpha,iter_step,recom_num=10):
    rank={}
    rank={point:0 for point in graph}  
    rank[root]=1 
    result={}
    for i in range(iter_step):
        tmp_rank={}   
        tmp_rank={point:0 for point in graph}
        for out_point,out_dict in graph.items(): 
            for inner_point,value in graph[out_point].items():
                tmp_rank[inner_point]+=alpha*rank[out_point]*graph[out_point][inner_point]
                if inner_point==root:
                    tmp_rank[inner_point]+=1-alpha
        if tmp_rank==rank:
            print("converged")
            break  
        rank=tmp_rank
    right_num=0  
    for instance in sorted(rank.items(),key=operator.itemgetter(1),reverse=True):
        point,pr_score=instance[0],instance[1]
        if point.split('_')[0] != 'artist':
            continue   
        if point in graph[root]:
            continue 
        result[point]=pr_score
        right_num+=1
        if right_num>=recom_num:   
            break
    return result    

In [18]:
#Example: Caluculate the recommendation result for user 8
graph = buildGrapha(df_train, tag_train, user_friends_df, k_friend, k_tag, k_artist, similarity)
alpha=0.6
user='8'
iter_num=100
recom_resul=personal_rank(graph,user,alpha,iter_num,20)
print(recom_resul)

converged
{'artist_289': 0.26579554098096797, 'artist_67': 0.09110335127858468, 'artist_701': 0.0711861437473364, 'artist_461': 0.06527547715682365, 'artist_55': 0.06396432918925081, 'artist_466': 0.0515463385993612, 'artist_257': 0.04747780413976701, 'artist_333': 0.044607007119820845, 'artist_498': 0.041091796454603736, 'artist_679': 0.040216629399768646, 'artist_294': 0.03382895681756284, 'artist_157': 0.031882693071810776, 'artist_302': 0.02843711671329293, 'artist_325': 0.02792381429026603, 'artist_306': 0.02446216352595119, 'artist_227': 0.02105021385303625, 'artist_299': 0.020985774285988704, 'artist_318': 0.02048224669327192, 'artist_72': 0.019176131523366592, 'artist_906': 0.019137861768216094}


### Based on Matrix

In [19]:
#Another efficient method to implement personal_rank algorithm

#Transform dictionary graph into matrix
def get_matrix_from_graph(graph):
  M=[]
  for point in graph.keys():
      row = []
      for next_point in graph.keys():
          if next_point in graph[point]:
              weight=graph[point][next_point]
              row.append(weight)
          else:
              row.append(0)
      M.append(row)
  return np.matrix(M)

#Transform matrix into csr matrix
def get_csr_matrix(M, alpha):
  n = M.shape[0]
  M1 = np.eye(n) - alpha * M.T
  data = []
  row_list = []
  col_list = []
  for row in range(n):
    for col in range(n):
      if (M1[row, col] != 0):
        data.append(M1[row, col])
        row_list.append(row)
        col_list.append(col)
  M = csr_matrix((data, (row_list, col_list)), shape=(n, n))
  return M

#Do personal_rank algorithm based on csr matrix
def personal_rank_based_on_matrix(graph, csr_matrix, user, alpha, recom_num=10):
  items=[]
  vertex=list(graph.keys())
  index=list(graph.keys()).index(user)
  n = csr_matrix.shape[0]
  zeros=np.zeros((n,1))
  zeros[index][0]=1
  r0=np.matrix(zeros)
  b = (1 - alpha) * r0
  res = gmres(csr_matrix, b, tol=1e-08)[0]
  tmp = {}
  result = {}
  for index in range(len(res)):
    point=vertex[index]
    tmp[point]=res[index]
  right_num=0  
  for instance in sorted(tmp.items(),key=operator.itemgetter(1),reverse=True):
      point,pr_score=instance[0],instance[1]
      if point.split('_')[0] != 'artist':
          continue   
      if point in graph[user]:
          continue 
      result[point]=pr_score
      right_num+=1
      if right_num>=recom_num:   
          break
  return result

## Example

In [34]:
#Define the possibility that the walker continues walking as 0.8
alpha = 0.6

#construct graph and transform it into sparse matrix
graph = buildGrapha_without_tag(df_train,  user_friends_df, k_friend, k_artist, similarity)
M = get_matrix_from_graph(graph)
csr_m = get_csr_matrix(M, alpha)

#get the user list to recommend
user_list = df_val['userID'].unique()

rc = {}
rc['userID'] = []
rc['recom_artist'] = []
rc['confidence'] = []
for user in user_list:
  str_user = str(user)
  result = personal_rank_based_on_matrix(graph, csr_m, str_user, alpha, recom_num=10)
  for artist,confidence in result.items():
    rc['userID'].append(user)
    artist = int(artist.split('_')[1])
    rc['recom_artist'].append(artist)
    rc['confidence'].append(confidence)
df = pd.DataFrame(rc)
df.to_csv('data/result/personalrank_without_tag.csv')

## Pickle Matrix

In [38]:
#The matrix and csr matrix has been saved in data/interim/matrix.pickle and data/interim/csr_matrix.pickle
with open('data/interim/matrix_without_tag.pickle', 'wb') as f:
  f.write(pickle.dumps(M))
with open('data/interim/csr_matrix_without_tag', 'wb') as f:
  f.write(pickle.dumps(csr_m))